<a href="https://colab.research.google.com/github/Angelvj/Alzheimer-disease-classification/blob/main/code/ad_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
kaggle = False

# Imports

In [ ]:
import os, shutil, re
import pickle
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import seaborn as sns
if kaggle:
    from kaggle_datasets import KaggleDatasets
    from kaggle_secrets import UserSecretsClient
else:
    from google.colab import drive
import nibabel as nib

# Import the most used layers
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Input, BatchNormalization, Dropout

# Hardware config.

In [ ]:
DEVICE = 'TPU' # or TPU
tpu = None

if DEVICE == 'TPU':
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        STRATEGY = tf.distribute.experimental.TPUStrategy(tpu)
    except ValueError:
        print('Could not connect to TPU, setting default strategy')
        tpu = None
        STRATEGY = tf.distribute.get_strategy()
elif DEVICE == 'GPU':
    STRATEGY = tf.distribute.MirroredStrategy()
    
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = STRATEGY.num_replicas_in_sync

print(f'Number of accelerators: {REPLICAS}')

Could not connect to TPU, setting default strategy
Number of accelerators: 1


# Functions for loading dataset

In [ ]:
def read_tfrecord(example):
    
    tfrec_format = {
        "image": tf.io.VarLenFeature(tf.float32),
        "one_hot_label": tf.io.VarLenFeature(tf.float32)
    }

    example = tf.io.parse_single_example(example, tfrec_format)
    one_hot_label = tf.sparse.to_dense(example['one_hot_label'])
    one_hot_label = tf.reshape(one_hot_label, [NUM_CLASSES])
    image = tf.reshape(tf.sparse.to_dense(example['image']), IMG_SHAPE)
    # TPU needs size to be known statically, so this doesn't work
    #     image  = tf.reshape(example['image'], example['shape']) 
    return image, one_hot_label

# =========================================================================
# The three functions below are used for generating a dataset directly from
# the filenames of the data (via a callable generator).
# ==========================================================================

def load_image(path):    

    img = nib.load(path)
    img = np.asarray(img.dataobj, dtype=np.float32)
    img = np.expand_dims(img, axis=3) # Add channel axis
    return img

def parse_file(filename, label):

    image = load_image(filename)
    image = np.nan_to_num(image, copy=False)
    label = np.eye(3, dtype=np.float32)[label]

    return image, label

def generator_fn(filenames, labels):

    def images_generator():

        for X, y in zip(filenames, labels):
            X, y = parse_file(X, y)
            yield X, y
            
    return images_generator

def load_dataset(filenames, labels, use_tfrec, no_order=True):
    
    if use_tfrec:
        # Allow order-altering optimizations
        option_no_order = tf.data.Options()
        option_no_order.experimental_deterministic = False
        dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
        if no_order:
            dataset = dataset.with_options(option_no_order)
        dataset = dataset.map(read_tfrecord, num_parallel_calls = AUTO)

    else:
        dataset = tf.data.Dataset.from_generator(generator_fn(filenames, labels),
            output_signature=(
                 tf.TensorSpec(shape=IMG_SHAPE, dtype=tf.float32),
                 tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.float32)))

    return dataset

def count_data_items(filenames, use_tfrec):
    
    if use_tfrec:
        n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
            for filename in filenames]
        return np.sum(n)
    else:
        return len(filenames)

def get_dataset(filenames, labels=None, use_tfrec=True, batch_size = 4, train=False, augment=False, cache=True, no_order=True):
    
    dataset =  load_dataset(filenames, labels, use_tfrec, no_order)
    
    if cache:
        dataset = dataset.cache() # Do it only if dataset fits in ram
    if train:
        dataset = dataset.repeat()
        if augment:
            raise NotImplementedError
#             dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
        dataset = dataset.shuffle(count_data_items(filenames, use_tfrec))

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTO)
    return dataset

# Train schedule

**Note**: not used yet (will be used when pretrain)

In [ ]:
# Only for pretraining
def get_lr_callback(batch_size=4, verbose=False):
    lr_start = 0.00001
    lr_max = 0.00004 * batch_size
    lr_min = 0.00001
    lr_rampup_epochs = 3
    lr_sustain_epochs = 0
    lr_exp_decay = 0.7

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = verbose)
    return lr_callback

# For training from scratch
def get_lr_decay_callback(batch_size=4, verbose=False):
    lr_max = 0.00001
    lr_exp_decay = 0.9
    lr_min = 0.000001
    
    def lrfn(epoch):
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch) + lr_min
        return lr
        
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = verbose)
    return lr_callback

# Build models

In [ ]:
# TODO (not important): define models in another file
def build_model_0(input_shape):
    
    inputs = tf.keras.layers.Input(input_shape)
    
    x = tf.keras.layers.Conv3D(filters=32, kernel_size=5, activation="relu")(inputs)
    x = MaxPooling3D(pool_size=2)(x)
    
    x = Flatten()(x)
    x = Dense(units=256, activation="relu")(x)
    
    outputs = Dense(units=3, activation="softmax")(x)
   
    model = tf.keras.Model(inputs, outputs, name="model_0")
    return model

def build_model_1(input_shape):
    
    inputs = tf.keras.layers.Input(input_shape)
    
    x = Conv3D(filters=32, kernel_size=5, activation="relu")(inputs)
    x = MaxPooling3D(pool_size=2)(x)
    
    x = Conv3D(filters=32, kernel_size=5, activation="relu")(x)
    x = MaxPooling3D(pool_size=2)(x)
    
    x = Flatten()(x)
    x = Dense(units=256, activation="relu")(x)
    
    outputs = Dense(units=3, activation="softmax")(x)
    
    model = tf.keras.Model(inputs, outputs, name="model_1")
    return model

def build_model_2(input_shape):
    
    inputs = tf.keras.layers.Input(input_shape)

    x = Conv3D(filters=16, kernel_size=5, activation="relu")(inputs)
    x = MaxPooling3D(pool_size=2)(x)

    x = Conv3D(filters=64, kernel_size=5, activation="relu")(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = Conv3D(filters=128, kernel_size=5, activation="relu")(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = Flatten()(x)
    x = Dense(units=256, activation="relu")(x)

    outputs = Dense(units=3, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs, name="model_2")
    return model

def build_model_3(input_shape):
    
    inputs = tf.keras.layers.Input(input_shape)

    x = Conv3D(filters=16, kernel_size=3, activation="relu")(inputs)
    x = Conv3D(filters=16, kernel_size=3, activation='relu')(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = BatchNormalization()(x)
    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = Flatten()(x)
    # x = Dropout(rate=0.1)(x)
    x = Dense(units=256, activation="relu")(x)

    outputs = Dense(units=3, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs, name="model_3")
    return model


def build_model_4(input_shape):
    inputs = tf.keras.layers.Input(input_shape)

    x = Conv3D(filters=16, kernel_size=3, activation="relu")(inputs)
    x = Conv3D(filters=16, kernel_size=3, activation='relu')(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=64, kernel_size=3, activation='relu')(x)
    x = Conv3D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = BatchNormalization()(x)
    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = Conv3D(filters=128, kernel_size=3, activation='relu')(x)
    x = Conv3D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling3D(pool_size=2)(x)

    x = Flatten()(x)
    # x = Dropout(rate=0.1)(x)
    x = Dense(units=256, activation="relu")(x)

    outputs = Dense(units=3, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs, name="model_4")
    return model

# Visualization

In [ ]:
def plot_epochs_history(num_epochs, history):

    plt.figure(figsize=(15, 5))
    plt.plot(np.arange(num_epochs), history['accuracy'], '-o', label='Train acc',
            color = '#ff7f0e')
    plt.plot(np.arange(num_epochs), history['val_accuracy'], '-o', label='Val acc',
            color = '#1f77b4')
    x = np.argmax(history['val_accuracy']); y = np.max(history['val_accuracy'])
    xdist = plt.xlim()[1] - plt.xlim()[0]; ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x, y, s=150, color='#1f77b4'); plt.text(x-0.03*xdist,y-0.13*ydist,'max acc\n%.2f'%y,size=14)
    plt.ylabel('ACC', size=14); plt.xlabel('Epoch', size=14)
    plt.legend(loc=2)
    plt2 = plt.gca().twinx()
    plt2.plot(np.arange(num_epochs),history['loss'],'-o',label='Train Loss',color='#2ca02c')
    plt2.plot(np.arange(num_epochs),history['val_loss'],'-o',label='Val Loss',color='#d62728')
    x = np.argmin(history['val_loss'] ); y = np.min(history['val_loss'])
    ydist = plt.ylim()[1] - plt.ylim()[0]
    plt.scatter(x,y,s=150,color='#d62728'); plt.text(x-0.03*xdist,y+0.05*ydist,'min loss',size=14)
    plt.ylabel('Loss',size=14)
    plt.legend(loc=3)
    plt.show()  
    
def plot_cm(labels, predictions):
    
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

# Functions for training and evaluating models

**Note**: bug in tf 2.4.1 (accuracies showed while training is different than the ones returned in history). [See issue here](https://github.com/tensorflow/tensorflow/issues/48033)

In [ ]:
def evaluate_model_kfold(model_builder, train_filenames, n_folds, batch_size, epochs, 
                         plot_fold_results = True, plot_avg_results = True, train_labels=None, 
                         stratify=False, shuffle=True, random_state=None, use_tfrec=True):
    
    # np_rs = np.random.RandomState(np.random.MT19937(np.random.SeedSequence(random_state)))
    folds_histories = []

    if stratify:
        skf = StratifiedKFold(n_splits=n_folds, shuffle=shuffle, random_state=random_state)
    else:
        skf = KFold(n_splits=n_folds, shuffle=shuffle, random_state=random_state)

    for fold, (idx_train, idx_val) in enumerate(skf.split(train_filenames, train_labels)):
        if tpu != None:
            tf.tpu.experimental.initialize_tpu_system(tpu)

        # np_rs.shuffle(idx_train)
        X_train = train_filenames[idx_train]
        X_val = train_filenames[idx_val]
        y_train = None if use_tfrec is None else train_labels[idx_train]
        y_val = None if use_tfrec is None else train_labels[idx_val]

        # Build model
        tf.keras.backend.clear_session()
        with STRATEGY.scope():
            model = model_builder(input_shape=IMG_SHAPE)
            # Optimizers and Losses create TF variables --> should always be initialized in the scope
            OPT = tf.keras.optimizers.Adam(learning_rate=LR)
            LOSS = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.00)
            model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS, steps_per_execution=8)

        # Train
        print(f'Training for fold {fold + 1} of {n_folds}...')
        cbks = [get_lr_decay_callback(batch_size)] # TODO: poner bien
        history = model.fit(
            get_dataset(X_train, y_train,  use_tfrec, train=True, augment=False, batch_size=batch_size), 
            epochs = EPOCHS, callbacks = cbks,
            steps_per_epoch = max(1, int(np.rint(count_data_items(X_train, use_tfrec)/batch_size))),
            validation_data = get_dataset(X_val, y_val, use_tfrec, batch_size = batch_size, train=False), 
            validation_steps= max(1, int(np.rint(count_data_items(X_val,use_tfrec)/batch_size))))
    
        if tf.__version__ == "2.4.1": # TODO: delete when tensorflow fixes the bug
            scores = model.evaluate(get_dataset(X_train, y_train, use_tfrec, batch_size = batch_size, train=False), 
                                    batch_size = batch_size, steps = max(1, int(np.rint(count_data_items(X_train, use_tfrec)/batch_size))))
            for i in range(len(model.metrics_names)):
                history.history[model.metrics_names[i]][-1] = scores[i]
            
        folds_histories.append(history.history)
        
        if plot_fold_results:
            plot_epochs_history(epochs, history.history)
        
    avg_history = avg_results_per_epoch(folds_histories)
            
    if plot_avg_results:
        
        plot_epochs_history(epochs, avg_history)

        print('-'*80)
        print('Results per fold')
        for i in range(n_folds):
            print('-'*80)
            out = f"> Fold {i + 1} - loss: {folds_histories[i]['loss'][-1]} - accuracy: {folds_histories[i]['accuracy'][-1]}"
            out += f" - val_loss.: {folds_histories[i]['val_loss'][-1]} - val_accuracy: {folds_histories[i]['val_accuracy'][-1]}"
            print(out)

        print('-'*80)
        print('Average results over folds (on last epoch):')
        print(f"> loss: {avg_history['loss'][-1]}")
        print(f"> accuracy: {avg_history['accuracy'][-1]}")
        print(f"> cval_loss: {avg_history['val_loss'][-1]}")
        print(f"> cval_accuracy: {avg_history['val_accuracy'][-1]}")
        print('-'*80)

    return folds_histories

def repeated_kfold(model_builder, train_filenames, n_folds, batch_size, epochs, reps=5, train_labels=None,
                   stratify=True, shuffle=True, random_state=None, use_tfrec=True):
    
    reps_histories = []
    
    for i in range(reps):
        print(f'Repetition {i + 1}')
        folds_histories = evaluate_model_kfold(model_builder, train_filenames, n_folds,
                                             batch_size, epochs, train_labels=train_labels, stratify=stratify,
                                             shuffle=shuffle, random_state=random_state, use_tfrec=use_tfrec)

        reps_histories.append(folds_histories)

    return reps_histories

def test_model_rkfold(model_builder, results_filename):
    # Evaluate model with repeated k-fold (because of the high variance)
    reps_results = repeated_kfold(model_builder, X_train, FOLDS, BATCH_SIZE, EPOCHS, reps=REPS, train_labels=y_train,
                   random_state=SEED)
    
    # Save results to disk
    f = open(results_filename, 'w' )
    f.write(repr(reps_results))
    f.close()
    
def train_model(model_builder):
    # Test model
    with STRATEGY.scope():
        OPT = tf.keras.optimizers.Adam(learning_rate=LR)
        LOSS = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.00)
        model = model_builder(IMG_SHAPE)
        model.compile(optimizer=OPT, loss=LOSS, metrics=METRICS)


    cbks = [get_lr_decay_callback(BATCH_SIZE)] # TODO: poner bien

    history = model.fit(
        get_dataset(X_train, train=True, batch_size=BATCH_SIZE), 
        epochs = EPOCHS, callbacks = cbks,
        steps_per_epoch = int(np.rint(count_data_items(X_train, use_tfrec=True)/BATCH_SIZE))
        )
    
    return model
    
def avg_results_per_epoch(histories):
    
    keys = list(histories[0].keys())
    epochs = len(histories[0][keys[0]])
    
    avg_histories = dict()
    for key in keys:
        avg_histories[key] = [np.mean([x[key][i] for x in histories]) for i in range(epochs)]
        
    return avg_histories

def avg_reps_results(reps_histories):
    return avg_results_per_epoch([avg_results_per_epoch(history) for history in reps_histories])
    
def show_rkfold_results(results_file):
    # Load results from disk
    f = open(results_file, 'r')
    reps_results = eval(f.read())
    
    reps_avgd_per_kfold = [avg_results_per_epoch(history) for history in reps_results]
    reps_avg = avg_results_per_epoch(reps_avgd_per_kfold)
    
    # Plot final result over epochs
    plot_epochs_history(EPOCHS, reps_avg)
    
    print('-'*80)
    print('Results per repetition (on last epoch)')
    for i in range(REPS):
        print('-'*80)
        print(f"> Repetition {i + 1} - Loss: {reps_avgd_per_kfold[i]['val_loss'][-1]} - Accuracy : {reps_avgd_per_kfold[i]['val_accuracy'][-1]}")

    print('-'*80)
    print('Average results over repetitions (on last epoch):')
    print(f"> Train Accuracy: {reps_avg['accuracy'][-1]}")
    print(f"> Train Loss: {reps_avg['loss'][-1]}")
    print(f"> CV accuracy: {reps_avg['val_accuracy'][-1]}")
    print(f"> CV Loss: {reps_avg['val_loss'][-1]}")
    print('-'*80)
    
def show_test_results(model):

    predictions = model.predict(get_dataset(X_test, no_order=False))
    y_pred = np.argmax(predictions, axis=1)

    plot_cm(y_test, y_pred)
    print(classification_report(y_test, y_pred))

def get_predictions(model, X):
    predict_proba = model.predict(get_dataset(X, no_order=False))
    y_pred = np.argmax(predict_proba, axis=1)
    return y_pred

def calculate_results(y_true, y_pred, labels = [0, 1, 2]):

    results = dict()
    results['sensitivity'] = dict()
    results['specificity'] = dict()

    cm = confusion_matrix(y_test, y_pred, labels)

    for i in range(len(cm)):

        true_positives = cm[i, i]
        false_positives_idx = [j for j in range(len(cm)) if j != i]
        false_positives = np.sum(cm[false_positives_idx, i])

        true_negatives = 0
        for j in range(len(cm)):
            for k in range(len(cm)):
                if j != i and k != i:
                    true_negatives += cm[j,k]

        false_negatives_idx = [j for j in range(len(cm)) if j != i]
        false_negatives = np.sum(cm[i, false_negatives_idx])

        sensitivity = true_positives/(true_positives + false_negatives)
        specificity = true_negatives/(true_negatives + false_positives)

        results['sensitivity'][labels[i]] = sensitivity
        results['specificity'][labels[i]] = specificity

    results['cm'] = cm
    results['accuracy'] = (true_positives + true_negatives)/np.sum(cm)

    return results


def present_results(results_dict, class_names):

    print('Accuracy:', results_dict['accuracy'], '\n')
    for c in class_names:
        print(f'---------- Results for class {class_names[c]} ----------')
        print(f" - Sensitivity: {results_dict['sensitivity'][c]}")
        print(f" - Specificity: {results_dict['specificity'][c]}\n")

    print(" --------- Confusion matrix --------- \n")

    disp = ConfusionMatrixDisplay(results_dict['cm'], ['NOR', 'AD', 'MCI'])
    disp.plot(cmap=plt.cm.Blues)

def save_dict(dic, filename):
    f = open(filename,'w')
    f.write(str(dic))
    f.close()

def load_dict(filename):
    f = open(filename,'r')
    data=f.read()
    data = data.replace('array', 'np.array')
    f.close()
    return eval(data)

# Prepare datasets and evaluation constants

In [ ]:
SEED = 268 # Arbitrary seed

# Name of different datasets used
TFREC_DATASETS = ['tfrec-pet-spatialnorm-elastic',
                  'tfrec-pet-spatialnorm-elastic-maxintensitynorm',
                  'tfrec-pet-spatialnorm-elastic-standarized',
                  'tfrec-pet-spatialnorm-rigid',
                  'tfrec-pet-spatialnorm-rigid-standarized',
                  'tfrec-pet-spatialnorm-regid-maxintensitynorm-standarized'
                 ]

# IMG_DATASETS = ['ad-preprocessed', None, None]

# Shape of images on each dataset
SHAPES = [(79, 95, 68, 1), (79, 95, 68, 1), (79, 95, 68, 1), (160, 160, 96, 1),
          (160, 160, 96, 1), (160, 160, 96, 1)]

# Number of repetitions and folds for repeated k-fold
REPS = 5
FOLDS = 10

# Different classes on the dataset
CLASSES = ['NOR', 'AD', 'MCI']

NUM_CLASSES = len(CLASSES)

METRICS = ['accuracy']

if kaggle:
    INPUT_DATAPATH = '/kaggle/input/' if tpu is None else None
    METADATA_PATH = '/kaggle/input/'
else:
    drive.mount('/content/drive') 
    INPUT_DATAPATH = '/content/drive/MyDrive/data/'
    METADATA_PATH = '/content/drive/MyDrive/data/'

Mounted at /content/drive


# Training/testing with PET images

## Spatially normalized PET - elastic transformations

In [ ]:
# # ========================  NOT USED =========================
# # Training model from images directly. Note: USE ONLY WITH GPU

# DS = IMG_DATASETS[0] # Select dataset
# IMG_SHAPE = SHAPES[0]
# DS_PATH =  INPUT_DATAPATH + DS

# # Path to images
# pet_paths = np.empty((0,), dtype=str)
# pet_labels = np.empty((0,), dtype=np.int64)

# for label, c in enumerate(CLASSES):
#     pattern = os.path.join(DS_PATH, c, 'PET') + '/*.nii'
#     pet_paths = np.concatenate((pet_paths, np.array(tf.io.gfile.glob(pattern))))
#     pet_labels = np.concatenate((pet_labels, np.full(len(pet_paths) - len(pet_labels), label, dtype=np.int64)))
    
# X_train, X_test, y_train, y_test = train_test_split(pet_paths, pet_labels, 
#                                                     stratify = pet_labels,
#                                                     test_size = 0.2,
#                                                     shuffle=True,
#                                                     random_state=SEED)

# LR = 0.00001
# BATCH_SIZE = 4 
# EPOCHS = 50

# reps_results = repeated_kfold(build_model_2, X_train, FOLDS, BATCH_SIZE, EPOCHS, reps=REPS, train_labels=y_train,
#                random_state=SEED, use_tfrec=False)

# reps_avgd_per_kfold = [avg_results_per_epoch(history) for history in reps_results]
# reps_avg = avg_results_per_epoch(reps_avgd_per_kfold)

# # Plot final result over epochs
# plot_epochs_history(EPOCHS, reps_avg)

# print('-'*80)
# print('Results per repetition (on last epoch)')
# for i in range(REPS):
#     print('-'*80)
#     print(f"> Repetition {i + 1} - Loss: {reps_avgd_per_kfold[i]['val_loss'][-1]} - Accuracy : {reps_avgd_per_kfold[i]['val_accuracy'][-1]}")

# print('-'*80)
# print('Average results over repetitions (on last epoch):')
# print(f"> Train Accuracy: {reps_avg['accuracy'][-1]}")
# print(f"> Train Loss: {reps_avg['loss'][-1]}")
# print(f"> CV accuracy: {reps_avg['val_accuracy'][-1]}")
# print(f"> CV Loss: {reps_avg['val_loss'][-1]}")
# print('-'*80)

In this case we have too little data and we need to reduce the variance of models evaluation, so we're going to use TFRecords "wrong" so that we can implement repeated-k-fold cross-validation (we have only one tfrecord per image, so we are not getting a good performance from TPU).

When we have more data, each tfrecord will have more than one example.

In [ ]:
def select_dataset(ds_id):
    global DS, IMG_SHAPE, DS_PATH, INPUT_DATAPATH
    DS = TFREC_DATASETS[ds_id]
    IMG_SHAPE = SHAPES[ds_id]
    if INPUT_DATAPATH == None:
        user_secrets = UserSecretsClient()
        user_credential = user_secrets.get_gcloud_credential()
        user_secrets.set_tensorflow_credential(user_credential)
        DS_PATH = KaggleDatasets().get_gcs_path(DS)
    else:
        DS_PATH = INPUT_DATAPATH + DS

In [ ]:
# Train from tfrecords: can be used with GPU or TPU

# =============================================================================
# "Load" data (get paths to train and test data)
# =============================================================================

select_dataset(0) # Not intensity normalized

metadata_train = pd.read_csv(METADATA_PATH + DS + '/train/train_summary.csv', encoding='utf-8')
metadata_test = pd.read_csv(METADATA_PATH + DS + '/test/test_summary.csv', encoding='utf-8')

X_train = DS_PATH + '/train/' + metadata_train.iloc[:, 0].to_numpy()
y_train = np.argmax(metadata_train.iloc[:,-len(CLASSES):].to_numpy(), axis=1)
X_test = DS_PATH + '/test/' + metadata_test.iloc[:, 0].to_numpy()
y_test = np.argmax(metadata_test.iloc[:,-len(CLASSES):].to_numpy(), axis=1)

# # =============================================================================
# # Train and test model 2 (and save results)
# # =============================================================================
# # Select hyperparameters
# LR = 0.00001 #, 0.00008
# BATCH_SIZE = 4 #, 64
# EPOCHS = 50

# test_model_rkfold(build_model_2, 'pet-spatialnorm-elastic_model-2_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic_model-2_results.txt')
# model = train_model(build_model_2)
# # model = tf.keras.models.load_model('pet-spatialnorm-elastic_model-2.h5') # Not used
# show_test_results(model)
# model.save('pet-spatialnorm-elastic_model-2.h5')

# # =============================================================================
# # Train and test model 0 (and save results)
# # =============================================================================
# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_0, 'pet-spatialnorm-elastic_model-0_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic_model-0_results.txt')
# model = train_model(build_model_0)
# show_test_results(model)
# model.save('pet-spatialnorm-elastic_model-0.h5')

# # =============================================================================
# # Train and test model 1 (and save results)
# # =============================================================================
# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_1, 'pet-spatialnorm-elastic_model-1_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic_model-1_results.txt')
# model = train_model(build_model_1)
# show_test_results(model)
# model.save('pet-spatialnorm-elastic_model-1.h5')

In [ ]:
select_dataset(1) # Max-intensity normalized

metadata_train = pd.read_csv(METADATA_PATH + DS + '/train/train_summary.csv', encoding='utf-8')
metadata_test = pd.read_csv(METADATA_PATH + DS + '/test/test_summary.csv', encoding='utf-8')

X_train = DS_PATH + '/train/' + metadata_train.iloc[:, 0].to_numpy()
y_train = np.argmax(metadata_train.iloc[:,-len(CLASSES):].to_numpy(), axis=1)
X_test = DS_PATH + '/test/' + metadata_test.iloc[:, 0].to_numpy()
y_test = np.argmax(metadata_test.iloc[:,-len(CLASSES):].to_numpy(), axis=1)

# # =============================================================================
# # Train and test model 2 (and save results)
# # =============================================================================
# # Select hyperparameters
# LR = 0.00001 #, 0.00008
# BATCH_SIZE = 4 #, 64
# EPOCHS = 50

# test_model_rkfold(build_model_2, 'pet-spatialnorm-elastic-maxintensitynorm_model-2_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-maxintensitynorm_model-2_results.txt')
# model = train_model(build_model_2)
# # model = tf.keras.models.load_model('pet-spatialnorm-elastic-maxintensitynorm_model-2.h5') # Not used
# show_test_results(model)
# model.save('pet-spatialnorm-elastic-maxintensitynorm_model-2.h5')

# # =============================================================================
# # Train and test model 0 (and save results)
# # =============================================================================
# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_0, 'pet-spatialnorm-elastic-maxintensitynorm_model-0_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-maxintensitynorm_model-0_results.txt')
# model = train_model(build_model_0)
# show_test_results(model)
# model.save('pet-spatialnorm-elastic-maxintensitynorm_model-0.h5')

# # =============================================================================
# # Train and test model 1 (and save results)
# # =============================================================================
# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_1, 'pet-spatialnorm-elastic-maxintensitynorm_model-1_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-maxintensitynorm_model-1_results.txt')
# model = train_model(build_model_1)
# show_test_results(model)
# model.save('pet-spatialnorm-elastic-maxintensitynorm_model-1.h5')

In [ ]:
select_dataset(2) # Standarized

metadata_train = pd.read_csv(METADATA_PATH + DS + '/train/train_summary.csv', encoding='utf-8')
metadata_test = pd.read_csv(METADATA_PATH + DS + '/test/test_summary.csv', encoding='utf-8')

X_train = DS_PATH + '/train/' + metadata_train.iloc[:, 0].to_numpy()
y_train = np.argmax(metadata_train.iloc[:,-len(CLASSES):].to_numpy(), axis=1)
X_test = DS_PATH + '/test/' + metadata_test.iloc[:, 0].to_numpy()
y_test = np.argmax(metadata_test.iloc[:,-len(CLASSES):].to_numpy(), axis=1)

# # =============================================================================
# # Train and test model 2 (and save results)
# # =============================================================================
# Select hyperparameters
# LR = 0.00001 #, 0.00008
# BATCH_SIZE = 4 #, 64
# EPOCHS = 50

# test_model_rkfold(build_model_2, 'pet-spatialnorm-elastic-standarized_model-2_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-standarized_model-2_results.txt')
# model = train_model(build_model_2)
# # model = tf.keras.models.load_model('pet-spatialnorm-elastic-standarized_model-2.h5') # Not used
# show_test_results(model)
# model.save('pet-spatialnorm-elastic-standarized_model-2.h5')

# # =============================================================================
# # Train and test model 0 (and save results)
# # =============================================================================
# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_0, 'pet-spatialnorm-elastic-standarized_model-0_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-standarized_model-0_results.txt')
# model = train_model(build_model_0)
# show_test_results(model)
# model.save('pet-spatialnorm-elastic-standarized_model-0.h5')

# # =============================================================================
# # Train and test model 1 (and save results)
# # =============================================================================
# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_1, 'pet-spatialnorm-elastic-standarized_model-1_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-standarized_model-1_results.txt')
# model = train_model(build_model_1)
# show_test_results(model)
# model.save('pet-spatialnorm-elastic-standarized_model-1.h5')


# # ====================== Deeper model ======================

# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_3, 'pet-spatialnorm-elastic-standarized_model-3_results.txt')
# show_rkfold_results('pet-spatialnorm-elastic-standarized_model-3_results.txt')
# # model = train_model(build_model_3)
# # show_test_results(model)
# # model.save('pet-spatialnorm-elastic-standarized_model-3.h5')


# ====================== Deeper model ======================

LR = 0.00001
BATCH_SIZE = 4
EPOCHS = 50

test_model_rkfold(build_model_4, 'pet-spatialnorm-elastic-standarized_model-4_results.txt')
show_rkfold_results('pet-spatialnorm-elastic-standarized_model-4_results.txt')
model_4 = train_model(build_model_4)

y_pred = get_predictions(model_4, X_test)
results = calculate_results(y_test, y_pred)
save_dict(results, 'pet-model_4-results.txt')
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})
model.save('pet-spatialnorm-elastic-standarized_model-4.h5')


In [ ]:
if tf.io.gfile.exists('Alzheimer-disease-classification'):
    shutil.rmtree('Alzheimer-disease-classification')
! git clone https://github.com/Angelvj/Alzheimer-disease-classification.git

results_path = 'Alzheimer-disease-classification/test_results/'


results = 0
results = load_dict(results_path + 'ensemble-results.txt')
present_results(results, {0:'NOR', 1:'AD', 2:'MCI'})

In [ ]:
# select_dataset(4)

# metadata_train = pd.read_csv(METADATA_PATH + DS + '/train/train_summary.csv', encoding='utf-8')
# metadata_test = pd.read_csv(METADATA_PATH + DS + '/test/test_summary.csv', encoding='utf-8')

# X_train = DS_PATH + '/train/' + metadata_train.iloc[:, 0].to_numpy()
# y_train = np.argmax(metadata_train.iloc[:,-len(CLASSES):].to_numpy(), axis=1)
# X_test = DS_PATH + '/test/' + metadata_test.iloc[:, 0].to_numpy()
# y_test = np.argmax(metadata_test.iloc[:,-len(CLASSES):].to_numpy(), axis=1)

# LR = 0.00001
# BATCH_SIZE = 4
# EPOCHS = 50

# test_model_rkfold(build_model_4, 'pet-spatialnorm-rigid-standarized_model-4_results.txt')
# show_rkfold_results('pet-spatialnorm-rigid-standarized_model-4_results.txt')
# # model = train_model(build_model_3)
# # show_test_results(model)
# # model.save('pet-spatialnorm-elastic-standarized_model-3.h5')


# K-fold results visualization

In [ ]:
if tf.io.gfile.exists('Alzheimer-disease-classification'):
    shutil.rmtree('Alzheimer-disease-classification')
! git clone https://github.com/Angelvj/Alzheimer-disease-classification.git

results_path = 'Alzheimer-disease-classification/kfold_results/'

EPOCHS = 50
REPS = 5

# # Model 0 results
# print(' ===================== Results for model 0 =====================')
# print(' - Spatially normalized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic_model-0_results.txt')
# print(' - Spatially normalized and max intensity normalized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-maxintensitynorm_model-0_results.txt')
# print(' - Spatially normalized and standarized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-standarized_model-0_results.txt')

# # Model 1 results
# print(' ===================== Results for model 1 =====================')
# print(' - Spatially normalized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic_model-1_results.txt')
# print(' - Spatially normalized and max intensity normalized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-maxintensitynorm_model-1_results.txt')
# print(' - Spatially normalized and standarized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-standarized_model-1_results.txt')

# # Model 2 results
# print(' ===================== Results for model 2 =====================')
# print(' - Spatially normalized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic_model-2_results.txt')
# print(' - Spatially normalized and max intensity normalized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-maxintensitynorm_model-2_results.txt')
# print(' - Spatially normalized and standarized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-standarized_model-2_results.txt')


# # Model 3 results
# print(' ===================== Results for model 3 =====================')
# print(' - Spatially normalized and standarized PET images')
# show_rkfold_results(results_path + 'pet-spatialnorm-elastic-standarized_model-3_results.txt')